# Purpose
This module is designed for reading in marker and dye data file.

The data shall be sotred in ".csv" format.

## Dyes
### dye_excitation.csv
spectral data of excitation of each dye, from 300 to 999 nm (step 1 nm).

### dye_emission.csv
spectral data of emission of each dye, from 300 to 999 nm (step 1 nm).

## Markers
### antigens.csv
first column as the symbol of each biomarker.

second column as each correlated antigen, 

thrid column as the relative brightness. \[level 1 to 5\]

In [1]:
using DataFrames
using CSV
using Query

In [12]:
readin_marker_density = [
    (:B220,10000), 
    (:IgD,20,:biotin), 
    (:Bcl_6,4000),
    (:GL_7,10000),
    (:FAS,3000),
]

laser_names = [:red, :yellow, :blue, :violet]

lasers = Dict(
    :red => 640,
    :yellow => 561,
    :blue => 488,
    :violet => 405
)

lens = Dict(
    :red    => [(660, 20), (730, 45), (780, 60)],
    :yellow => [(586, 15), (610, 20), (670, 30), (710, 50), (780, 60)],
    :blue   => [(530, 30), (575, 26), (610, 20), (670, 30), (710, 50), (780, 60)],
    :violet => [(450, 50), (510, 50), (605, 40), (660, 40)]
)

Dict{Symbol,Array{Tuple{Int64,Int64},1}} with 4 entries:
  :red    => Tuple{Int64,Int64}[(660, 20), (730, 45), (780, 60)]
  :yellow => Tuple{Int64,Int64}[(586, 15), (610, 20), (670, 30), (710, 50), (78…
  :violet => Tuple{Int64,Int64}[(450, 50), (510, 50), (605, 40), (660, 40)]
  :blue   => Tuple{Int64,Int64}[(530, 30), (575, 26), (610, 20), (670, 30), (71…

## import CSV

In [4]:
excitation = CSV.read("../data/常见染料_发射曲线.csv",
    #types=[Int,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64],
    allowmissing=:none
)

emission = CSV.read("../data/常见染料_激发曲线.csv",
    #types=[Int,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64],
    allowmissing=:none
)

antigens = CSV.read("../data/使用抗体.csv", allowmissing=:none)

head(antigens)

,markers,dyes,brightness
,String,String,Int64
1,Bcl_6,Alexa647,4
2,CD11c,Alexa647,4
3,GL_7,Alexa647,4
4,CD8alpha,Alexa700,1
5,B220,APC,5
6,CD3,APC,5


### extract names of dyes and markers

In [5]:
dyes_em = [item for item in getfield(emission, :colindex).names if item != :Wavelength && item !=:wavelength] |> sort!
dyes_ex = [item for item in getfield(excitation, :colindex).names if item != :Wavelength && item !=:wavelength] |> sort!

if dyes_em == dyes_ex
    dyes = dyes_ex
else
    println("names of dyes not match!")
    # TODO: using logging module
end
markers = [Symbol(item) for item in antigens[:markers]] |> unique

41-element Array{Symbol,1}:
 :Bcl_6     
 :CD11c     
 :GL_7      
 :CD8alpha  
 :B220      
 :CD3       
 :CD45_1    
 :CD45_2    
 :CD8       
 :CD80      
 :F4_80     
 :FAS       
 :IgG       
 ⋮          
 :TCR_beta  
 :Thy1_2    
 :AnnexinV  
 :CD138     
 :CD38      
 :CD86      
 :CXCR4     
 :TCRValpha2
 :Thy1_1    
 :PD_1      
 :PDL_1     
 :CD205     

## get dye brightness constant

In [58]:
function getDyeBrightness(antigen_list::DataFrame, markername::Symbol, dyename::Symbol)
    B = @from item in antigen_list begin
        @where item.markers == String(markername) && item.dyes == String(dyename)
        @select item.brightness
        @collect
    end
    
    if length(B) == 1
        return B[1]
    elseif length(B) > 1
        error("duplicated marker-dye pair for $(markername)-$(dyename)")
    else
#         error("not found marker-dye pair for $(markername)-$(dyename)")
        return NaN
    end
    
end

dye_brightness = Dict{Tuple{Symbol, Symbol}, Union{Int, Float64}}()
for j in dyes, i in readin_marker_density
    dye_brightness[(i[1],j)] = getDyeBrightness(antigens, i[1], j)
end
dye_brightness

Dict{Tuple{Symbol,Symbol},Union{Float64, Int64}} with 60 entries:
  (:GL_7, :BV510)       => NaN
  (:FAS, :BV510)        => NaN
  (:IgD, :PE)           => 5
  (:B220, :PE)          => 5
  (:IgD, :APC)          => NaN
  (:B220, :FITC)        => 2
  (:GL_7, :FITC)        => NaN
  (:IgD, :Alexa647)     => NaN
  (:Bcl_6, :PE)         => NaN
  (:FAS, :Alexa647)     => NaN
  (:FAS, :PacificBlue)  => NaN
  (:GL_7, :PE)          => 5
  (:Bcl_6, :BV605)      => NaN
  (:FAS, :APC)          => 5
  (:Bcl_6, :Alexa700)   => NaN
  (:Bcl_6, :FITC)       => NaN
  (:IgD, :FITC)         => 2
  (:FAS, :APC_Cy7)      => NaN
  (:IgD, :PE_Cy7)       => NaN
  (:B220, :Alexa700)    => NaN
  (:Bcl_6, :PE_Dazzle)  => NaN
  (:B220, :PacificBlue) => 2
  (:FAS, :Alexa700)     => NaN
  (:FAS, :BV605)        => NaN
  (:Bcl_6, :BV510)      => NaN
  ⋮                     => ⋮

In [61]:
function getExcitedRatio(dye_excitation::DataFrame, dyename::Symbol, lambda::Int64)
    E = @from item in dye_excitation begin
        @where item.Wavelength == lambda
        @select item[dyename]
        @collect
    end
    
    if length(E) == 1
        return E[1] |> log10
    elseif length(E) > 1
        error("duplicated excitation curve for dye: $(dyename)")
    else
        error("not found excitation curve for dye: $(dyename)")
    end
    
end

getExcitedRatio (generic function with 1 method)

In [66]:
excitratio = Dict{Tuple{Symbol, Symbol}, Float64}()
@time for laseridx in keys(lasers)
    for eachdye in dyes
        excitratio[(eachdye, laseridx)] = getExcitedRatio(excitation, eachdye, lasers[laseridx])
    end
end
excitratio;

  0.001999 seconds (2.44 k allocations: 167.031 KiB)


In [91]:
function getEmitRatio(dye_emission::DataFrame, dyename::Symbol, loc::Int, scale::Int)
    E = @from item in dye_emission begin
        @where loc-scale < item.Wavelength < loc+scale
        @select item[dyename]
        @collect
    end
    
    if length(E) >= 1
        return sum(E) .|> x->x<0 ? 0 : x |> log10
    else
        error("not found emission curve for dye: $(dyename)")
    end
    
end
getEmitRatio(emission, :APC, 660, 30)

3.4758372670873787

In [98]:
emitratio = Dict{Tuple{Symbol, Symbol, Int}, Any}()
@time for (laseridx, value) in lens
    for (eachloc, eachscale) in value
        for eachdye in dyes
            emitratio[(eachdye, laseridx, eachloc)] = getEmitRatio(emission, eachdye, eachloc, eachscale)
        end
    end
end
emitratio

  0.025483 seconds (46.99 k allocations: 3.191 MiB)


Dict{Tuple{Symbol,Symbol,Int64},Any} with 216 entries:
  (:APC, :yellow, 610)         => 3.39589
  (:Alexa700, :violet, 660)    => 3.69383
  (:APC_Cy7, :blue, 610)       => 3.37608
  (:PE_Dazzle, :violet, 510)   => 3.76187
  (:Alexa647, :blue, 780)      => -Inf
  (:BV605, :blue, 670)         => -Inf
  (:PE_Cy7, :violet, 660)      => 2.85196
  (:APC_Cy7, :blue, 670)       => 3.73486
  (:Alexa700, :blue, 780)      => 2.76238
  (:APC_Cy7, :blue, 530)       => 2.7142
  (:Alexa700, :blue, 575)      => 2.73567
  (:Alexa647, :violet, 450)    => -Inf
  (:APC_Cy7, :yellow, 780)     => 2.32747
  (:Alexa700, :violet, 510)    => 2.32833
  (:PE_Dazzle, :yellow, 670)   => 1.54362
  (:APC_Cy7, :yellow, 610)     => 3.37608
  (:BV605, :violet, 450)       => 3.26643
  (:FITC, :blue, 670)          => -Inf
  (:BV605, :blue, 530)         => 2.59905
  (:PE_Dazzle, :red, 730)      => 1.11651
  (:Alexa647, :violet, 660)    => 3.58631
  (:PE, :yellow, 710)          => -Inf
  (:Percp_Cy5_5, :violet, 510) => 3.9

In [ ]:
function getLensInteraction()
    
end

---

In [1]:
function getLensNumber(lens)
    count = 0
    for (name, port) in lens
        count += length(port)
    end
    count
end         

function generateLensInteraction(emit_ratio, excite_ratio, dye_list, laser_list, lens, 
        dye_brightness)
    
    result = Dict()

    for (each_laser, _) in laser_list
        laser_array = Dict()
        for (port, _) in lens[each_laser]
            # println(each_laser, " ", port)
            port_array = Dict()
            for dominator in dye_list
                domi_value = excite_ratio[each_laser][dominator][1] / 100 * 
                             mean(emit_ratio[each_laser][port][dominator]) *
                             dye_brightness[dominator]
                dye_array = []
                for liberator in dye_list
                    if liberator != dominator
                        libe_value = excite_ratio[each_laser][liberator][1] / 100 * 
                                     mean(emit_ratio[each_laser][port][liberator]) *
                                     dye_brightness[liberator]
                        push!(dye_array, (domi_value-libe_value)/(domi_value+libe_value+1))
                    else
                        push!(dye_array, -Inf) 
                    end
                    
                end
                port_array[dominator] = dye_array
            end
            laser_array[port]=port_array
        end
        result[each_laser] = laser_array
    end
    result
end

generateLensInteraction (generic function with 1 method)

$$ \frac{\alpha_{ex}}{100} \cdot \langle \alpha_{em} \rangle \cdot \alpha_L$$

In [33]:
demo = generateLensInteraction(
    getEmitRatio(lens, emission, dyes),
    getExcitedRatio(lasers, excitation, dyes),
    dyes, lasers, lens, dye_brightness
)

MethodError: MethodError: no method matching getEmitRatio(::Dict{Symbol,Array{Tuple{Int64,Int64},1}}, ::DataFrame, ::Array{Symbol,1})
Closest candidates are:
  getEmitRatio(::Any, ::Any) at In[30]:20

In [9]:
demo[:red][780][:BV605]

LoadError: [91mUndefVarError: demo not defined[39m

In [39]:
getEmitRatio(lens, emission)[:red][780]

┌ Warning: `find(f::Function, A::AbstractVector)` is deprecated, use `findall(f, A)` instead.
│   caller = getExcitedRatio(::Dict{Symbol,Array{Tuple{Int64,Int64},1}}, ::DataFrame) at In[30]:4
└ @ Main ./In[30]:4


BoundsError: BoundsError: attempt to access 0-element Array{Int64,1} at index [1]

---

In [11]:
function getTemplate(antigens, marker_list, dye_list, level=1)
    template = zeros(length(marker_list), length(dye_list))
    for i=1:length(antigens[:,1])
        a1 = indexin([antigens[i,:markers] |> Symbol], marker_list)[1]
        a2 = indexin([antigens[i,:dyes] |> Symbol], dye_list)[1]

        if a2 == 0 
            println([antigens[i,:dyes] |> Symbol])
        end

        template[a1, a2] = 1
    end
    template
    repeat(template, outer=(1,1,level))
end
getTemplate(antigens, markers, dyes, getLensNumber(lens))

41×12×18 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮       
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 